# 03 - Définition et calcul des KPI

A ce niveau:
- On charge les données **nettoyées** depuis le dossier **cleaned_data** dans une base SQLite en mémoire
- On définit **6 KPI** conformes au sujet (performance, accessibilité/couverture, qualité de service, efficience)

| Nom du KPI | Objectif métier | Description / Interprétation | Règle de calcul (explicite) | Requête SQL |
|------------|-----------------|------------------------------|-----------------------------|-------------|
| Délai moyen de traitement des demandes (jours) | Mesurer le délai moyen global de traitement des demandes de documents administratifs. | Indique le nombre moyen de jours nécessaires pour traiter une demande, en tenant compte du volume de demandes. Plus il est faible, plus le service est réactif. | **Source :** table `demandes_service_public`. 1) Pour chaque ligne, on considère le délai moyen de traitement `delai_traitement_jours` et le volume associé `nombre_demandes`. 2) On calcule la somme pondérée des délais : Σ(`nombre_demandes` × `delai_traitement_jours`). 3) On calcule le volume total Σ(`nombre_demandes`). 4) Le KPI est le rapport : **Σ(nombre_demandes × delai_traitement_jours) / Σ(nombre_demandes)**. | `SELECT SUM(nombre_demandes * delai_traitement_jours) * 1.0 / NULLIF(SUM(nombre_demandes), 0) AS delai_moyen_jours FROM demandes_service_public;` |
| Taux d’occupation moyen des centres (%) | Évaluer dans quelle mesure les capacités théoriques des centres sont utilisées. | Compare le volume effectivement traité à la capacité théorique (personnel_capacite_jour × nombre de jours d’activité). Un taux très faible = sous-utilisation, très élevé = risque de saturation. | **Source :** tables `logs_activite` et `centres_service`. 1) Pour chaque centre (`centre_id`), on compte le nombre de jours où il y a une activité dans `logs_activite` : `nb_jours = COUNT(DISTINCT date_operation)`. 2) On calcule le volume total traité par centre : `total_traite = Σ(nombre_traite)`. 3) On récupère la capacité théorique journalière de chaque centre : `personnel_capacite_jour` (colonne de `centres_service`). 4) On calcule la capacité théorique totale sur la période : Σ(`personnel_capacite_jour` × `nb_jours`). 5) Le KPI global est : **taux_occupation = Σ(total_traite) / Σ(personnel_capacite_jour × nb_jours) × 100**. | `WITH days_per_centre AS ( SELECT centre_id, COUNT(DISTINCT date_operation) AS nb_jours FROM logs_activite GROUP BY centre_id ), treated AS ( SELECT centre_id, SUM(COALESCE(nombre_traite, 0)) AS total_traite FROM logs_activite GROUP BY centre_id ) SELECT SUM(t.total_traite) * 100.0 / NULLIF(SUM(c.personnel_capacite_jour * d.nb_jours), 0) AS taux_occupation_moyen_pct FROM treated t JOIN days_per_centre d USING (centre_id) JOIN centres_service c USING (centre_id);` |
| Distance moyenne commune → centre le plus proche (km) | Mesurer l’accessibilité territoriale moyenne au réseau de centres. | Approxime, pour chaque commune, la distance au centre le plus proche, puis en fait la moyenne. Plus cette distance est faible, meilleure est l’accessibilité. | **Source :** tables `details_communes` et `centres_service`. 1) Pour chaque couple (commune, centre), on calcule une distance approximative en km à partir des coordonnées (`latitude`, `longitude`) : `distance_km ≈ sqrt((lat_commune − lat_centre)^2 + (lon_commune − lon_centre)^2) × 111`. 2) Pour chaque commune, on garde la distance minimale `distance_min_km` parmi tous les centres. 3) Le KPI est la moyenne de ces distances minimales : **distance moyenne = AVG(distance_min_km)**. | `WITH distances AS ( SELECT dc.commune AS commune, MIN( SQRT( (dc.latitude - cs.latitude) * (dc.latitude - cs.latitude) + (dc.longitude - cs.longitude) * (dc.longitude - cs.longitude) ) * 111.0 ) AS distance_min_km FROM details_communes dc CROSS JOIN centres_service cs GROUP BY dc.commune ) SELECT AVG(distance_min_km) AS distance_moyenne_km FROM distances;` |
| Taux de couverture territoriale (communes à ≤ 20 km) | Mesurer la part des communes « raisonnablement couvertes » par un centre de service. | Pourcentage de communes dont le centre le plus proche est à 20 km ou moins. Permet d’identifier les zones potentiellement sous-desservies. | **Source :** même sous-requête `distances` que pour le KPI précédent. 1) On dispose, pour chaque commune, de `distance_min_km` (distance au centre le plus proche). 2) On compte le nombre de communes avec `distance_min_km ≤ 20`. 3) On divise par le nombre total de communes. 4) Le KPI est : **taux_couverture = nb_communes(distance_min_km ≤ 20) / nb_total_communes** (que l’on peut exprimer ensuite en %). | `WITH distances AS ( SELECT dc.commune AS commune, MIN( SQRT( (dc.latitude - cs.latitude) * (dc.latitude - cs.latitude) + (dc.longitude - cs.longitude) * (dc.longitude - cs.longitude) ) * 111.0 ) AS distance_min_km FROM details_communes dc CROSS JOIN centres_service cs GROUP BY dc.commune ) SELECT SUM(CASE WHEN distance_min_km <= 20 THEN 1 ELSE 0 END) * 1.0 / COUNT(*) AS taux_couverture_20km FROM distances;` |
| Taux de rejet global des opérations | Suivre la qualité de service à travers la proportion de demandes/opérations rejetées. | Indique la part des volumes rejetés par rapport au total traité + rejeté. Un taux élevé signale des problèmes de qualité (dossiers incomplets, erreurs, règles peu claires…). | **Source :** table `logs_activite`. 1) Pour l’ensemble de la période et de tous les centres, on somme le volume rejeté : Σ(`nombre_rejete`). 2) On somme le volume total entrant : Σ(`nombre_traite` + `nombre_rejete`). 3) Le KPI est le rapport : **taux_rejet = Σ(nombre_rejete) / Σ(nombre_traite + nombre_rejete)**. | `SELECT SUM(COALESCE(nombre_rejete, 0)) * 1.0 / NULLIF(SUM(COALESCE(nombre_traite, 0) + COALESCE(nombre_rejete, 0)), 0) AS taux_rejet_global FROM logs_activite;` |
| Productivité moyenne des agents (documents traités par agent présent et par jour) | Mesurer l’efficience opérationnelle du personnel dans les centres. | Nombre moyen de documents traités par agent présent et par jour. Permet de comparer la productivité entre centres et d’identifier les bonnes pratiques ou les goulots d’étranglement. | **Source :** table `logs_activite`. 1) Pour chaque ligne de log (centre × date), on dispose du nombre de documents traités `nombre_traite` et du nombre d’agents présents `personnel_present`. 2) On somme les documents traités : Σ(`nombre_traite`). 3) On somme le nombre d’agents présents : Σ(`personnel_present`). 4) Le KPI est le rapport : **productivité = Σ(nombre_traite) / Σ(personnel_present)** (en « documents par agent et par jour observé »). | `SELECT SUM(COALESCE(nombre_traite, 0)) * 1.0 / NULLIF(SUM(COALESCE(personnel_present, 0)), 0) AS docs_par_agent_par_jour FROM logs_activite;` |